In [1]:
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display
from jetbot import Camera, bgr8_to_jpeg

camera = Camera.instance(width=224, height=224, usb=0)

image = widgets.Image(format='jpeg', width=400, height=400)  # this width and height doesn't necessarily have to match the camera

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)
classfication_result_image = widgets.Textarea(
            value=u'Result:=',
            description=u'Classifcation',
            disabled=False)



In [2]:
import cv2
import tensorflow as tf
import numpy as np

def rgba2rgb(rgba, background=(255,255,255)):
    row, col, ch = rgba.shape
    if ch == 3:
        return rgba
    assert ch == 4, "RGBA image has 4 channels"
    rgb = np.zeros((row, col, 3), dtype='float32')
    r, g, b, a = rgba[:,:,0], rgba[:,:,1], rgba[:,:,2], rgba[:,:,3]
    a = np.asarray(a, dtype='float32') / 255.0
    R, G, B = background
    rgb[:,:,0] = r * a + (1.0 - a) * R
    rgb[:,:,1] = g * a + (1.0 - a) * G
    rgb[:,:,2] = b * a + (1.0 - a) * B
    return np.asarray(rgb, dtype="uint8")

def preprocess(camera_value):
    data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)
    x = camera_value
    x = cv2.resize(x, (224, 224))
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = np.asarray(x)
    normalized_image_array = (x.astype(np.float32) / 127.0) - 1
    data[0] = normalized_image_array
    return data

In [3]:
model = tf.keras.models.load_model('keras_model.h5')
np.set_printoptions(suppress=True)


In [ ]:
#data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)
#image = preprocess(camera.value)
#image_array = np.asarray(image)
#normalized_image_array = (image_array.astype(np.float32) / 127.0) - 1

# Load the image into the array
#data[0] = normalized_image_array

# run the inference
#prediction = model.predict(data)
#print(prediction)

In [4]:
import numpy as np
data, target = np.array_split(np.loadtxt('labels.txt', dtype=str), [-1], axis=1)

label=target.flatten().tolist()

In [5]:
display(widgets.VBox([
    widgets.HBox([image, classfication_result_image]),
]))

In [6]:
def execute(change):
    image = change['new']
    classfication_result_image.value = ' '
    prediction = model.predict(preprocess(camera.value))
    a=prediction[0].argmax(0)
    classfication_result_image.value = label[a]
#    output = model_trt(preprocess(camera.value).half())[0].detach().cpu().float().numpy()
#    mask=decode_segmap(output.argmax(0))
    #mask = 1.0 * (output.argmax(0) == 15)
    #m = mask[:, :, None]
    #print(m.shape)
    #print(image.shape)
    #seg_image.value = bgr8_to_jpeg(mask[:, :, None] * image)
    #today = datetime.now()

    #classfication_result_image.value = "Text" + today.strftime("%d/%m/%Y %H:%M:%S")
    
mask = execute({'new': camera.value})
camera.observe(execute, names='value')

In [7]:
camera.stop()

In [4]:
from tensorflow.keras.models import load_model

from PIL import Image, ImageOps
import numpy as np

In [5]:

model = load_model('keras_model.h5')

In [8]:
import tensorflow as tf
converter = tf.lite.TFLiteConverter.from_keras_model_file('keras_model.h5') 


AttributeError: type object 'TFLiteConverterV2' has no attribute 'from_keras_model_file'

In [ ]:
tfmodel = converter.convert() 
open ('model.tflite' , "wb") .write(tfmodel)